In [4]:
import pickle

with open('dataframe.pkl', 'rb') as file:
    df = pickle.load(file)

In [5]:
from sklearn.model_selection import train_test_split

# Séparez les données en fonction de la variable 'Flagimpaye'
df_pas_utilise, df_reduit = train_test_split(df, test_size=0.01, stratify=df['FlagImpaye'], random_state=42)

# Affichez la taille du sous-DataFrame de test pour vérifier
print("Taille du sous-DataFrame de test :", len(df_reduit))

Taille du sous-DataFrame de test : 46468


In [6]:
df_reduit.info()

<class 'pandas.core.frame.DataFrame'>
Index: 46468 entries, 4207882 to 873112
Data columns (total 27 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   FlagImpaye       46468 non-null  category      
 1   Montant          46468 non-null  float64       
 2   DateTransaction  46468 non-null  datetime64[ns]
 3   CodeDecision     46468 non-null  object        
 4   VerifianceCPT1   46468 non-null  int32         
 5   VerifianceCPT2   46468 non-null  int32         
 6   VerifianceCPT3   46468 non-null  int32         
 7   D2CB             46468 non-null  int32         
 8   ScoringFP1       46468 non-null  float64       
 9   ScoringFP2       46468 non-null  float64       
 10  ScoringFP3       46468 non-null  float64       
 11  TauxImpNb_RB     46468 non-null  float64       
 12  TauxImpNB_CPM    46468 non-null  float64       
 13  EcartNumCheq     46468 non-null  int32         
 14  NbrMagasin3J     46468 non-null  int

### On définit la partie Train et Test

In [7]:
# Définition des dates limites pour les ensembles d'apprentissage et de test
train_inf = '2017-02-01'
train_sup = '2017-08-31'
test_inf = '2017-09-01'
test_sup = '2017-11-30'

In [8]:
train = df.loc[(df['DateTransaction'] >= train_inf) & (df['DateTransaction'] <= train_sup)]
X_train = train.drop(columns=['FlagImpaye','CodeDecision','DateTransaction'])
y_train = train['FlagImpaye']

In [9]:
test = df.loc[(df['DateTransaction'] >= test_inf) & (df['DateTransaction'] <= test_sup)]
X_test = test.drop(columns=['FlagImpaye','CodeDecision','DateTransaction'])
y_test = test['FlagImpaye']

In [10]:
len(X_train)

38827

In [11]:
len(X_test)

7441

## Test de chaque méthode d'échantillonage par modeles

In [12]:
from sklearn import metrics
from sklearn.metrics import f1_score

In [13]:
!pip install 'imblearn'
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN, BorderlineSMOTE

ERROR: Invalid requirement: "'imblearn'"

[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
!pip install xgboost


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


On fait tourner DecisionTreeClassifier, RandomForestClassifier, GradientBoostingClassifier,KNeighborsClassifier, SVC, adaboost et xgboost

In [2]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from imblearn.over_sampling import RandomOverSampler, SMOTE, BorderlineSMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from xgboost import XGBClassifier

In [15]:

# Définir les stratégies d'échantillonnage
sampling_strategies = [0.01, 0.05, 0.1]  # Ajoutez les valeurs que vous souhaitez tester
samplers = [RandomOverSampler, SMOTE, BorderlineSMOTE, ADASYN, RandomUnderSampler, NearMiss]

# Définir les modèles à tester
models = [
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    GradientBoostingClassifier(),
    KNeighborsClassifier(),
    SVC(),
    AdaBoostClassifier(),
    XGBClassifier()
    
]

# Initialiser un DataFrame pour stocker les résultats
results_df = pd.DataFrame(columns=['Model', 'Sampler', 'Sampling Strategy', 'F1 Score'])

# Parcourir chaque modèle
for model in models:
    # Parcourir chaque échantillonneur
    for sampler in samplers:
        # Parcourir chaque valeur de sampling_strategy
        for strategy in sampling_strategies:
            # Appliquer l'échantillonnage
            X_resampled, y_resampled = sampler(sampling_strategy=strategy).fit_resample(X_train, y_train)

            # Initialiser et entraîner le modèle
            model.fit(X_resampled, y_resampled)

            # Faire des prédictions sur l'ensemble de test
            y_pred = model.predict(X_test)

            # Évaluer les performances du modèle
            f1 = f1_score(y_test, y_pred)

            # Ajouter les résultats au DataFrame
            results_df = pd.concat([results_df, pd.DataFrame({
                'Model': [model.__class__.__name__],
                'Sampler': [sampler.__name__],
                'Sampling Strategy': [strategy],
                'F1 Score': [f1]
            })], ignore_index=True)


# Afficher le tableau récapitulatif
print(results_df)

C:\Users\marevel\AppData\Local\Temp\ipykernel_7464\963340370.py:50: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame({


                      Model             Sampler  Sampling Strategy  F1 Score
0    DecisionTreeClassifier   RandomOverSampler               0.01  0.013699
1    DecisionTreeClassifier   RandomOverSampler               0.05  0.078947
2    DecisionTreeClassifier   RandomOverSampler               0.10  0.017857
3    DecisionTreeClassifier               SMOTE               0.01  0.063348
4    DecisionTreeClassifier               SMOTE               0.05  0.032468
..                      ...                 ...                ...       ...
121           XGBClassifier  RandomUnderSampler               0.05  0.112676
122           XGBClassifier  RandomUnderSampler               0.10  0.075862
123           XGBClassifier            NearMiss               0.01  0.044944
124           XGBClassifier            NearMiss               0.05  0.050761
125           XGBClassifier            NearMiss               0.10  0.030033

[126 rows x 4 columns]


On réunit les résultats dans un seul tableau recapitulatif

In [17]:
results_df

,Model,Sampler,Sampling Strategy,F1 Score
0,DecisionTreeClassifier,RandomOverSampler,0.01,0.013699
1,DecisionTreeClassifier,RandomOverSampler,0.05,0.078947
2,DecisionTreeClassifier,RandomOverSampler,0.10,0.017857
3,DecisionTreeClassifier,SMOTE,0.01,0.063348
4,DecisionTreeClassifier,SMOTE,0.05,0.032468
...,...,...,...,...
121,XGBClassifier,RandomUnderSampler,0.05,0.112676
122,XGBClassifier,RandomUnderSampler,0.10,0.075862
123,XGBClassifier,NearMiss,0.01,0.044944
124,XGBClassifier,NearMiss,0.05,0.050761


In [18]:
results_df.describe(include='all')

,Model,Sampler,Sampling Strategy,F1 Score
count,126,126,126.000000,126.000000
unique,7,6,NaN,NaN
top,DecisionTreeClassifier,RandomOverSampler,NaN,NaN
freq,18,21,NaN,NaN
mean,NaN,NaN,0.053333,0.028053
std,NaN,NaN,0.036965,0.030843
min,NaN,NaN,0.010000,0.000000
25%,NaN,NaN,0.010000,0.000000
50%,NaN,NaN,0.050000,0.023739
75%,NaN,NaN,0.100000,0.047848


In [20]:
# Grouper les résultats par modèle
grouped_results = results_df.groupby('Model')

# Initialiser un DataFrame pour stocker les meilleurs résultats par modèle
best_results_df = pd.DataFrame(columns=['Model', 'Sampler', 'Sampling Strategy', 'F1 Score'])

# Parcourir chaque groupe
for model, group_df in grouped_results:
    # Trouver la ligne avec le plus grand F1 score pour chaque modèle
    best_result = group_df.sort_values(by='F1 Score', ascending=False).iloc[0]
    
    # Ajouter le meilleur résultat au DataFrame final
    # Remplacez la ligne best_results_df = best_results_df.append(...) par ceci
    best_results_df = pd.concat([best_results_df, pd.DataFrame({
        'Model': [best_result['Model']],
        'Sampler': [best_result['Sampler']],
        'Sampling Strategy': [best_result['Sampling Strategy']],
        'F1 Score': [best_result['F1 Score']]
    })], ignore_index=True)

# Afficher le tableau récapitulatif des meilleurs résultats pour chaque modèle
print(best_results_df)


                        Model             Sampler  Sampling Strategy  F1 Score
0          AdaBoostClassifier  RandomUnderSampler               0.05  0.116667
1      DecisionTreeClassifier   RandomOverSampler               0.05  0.078947
2  GradientBoostingClassifier  RandomUnderSampler               0.10  0.124352
3        KNeighborsClassifier               SMOTE               0.05  0.036036
4      RandomForestClassifier  RandomUnderSampler               0.10  0.098361
5                         SVC   RandomOverSampler               0.01  0.000000
6               XGBClassifier  RandomUnderSampler               0.05  0.112676


C:\Users\marevel\AppData\Local\Temp\ipykernel_7464\2611653697.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  best_results_df = pd.concat([best_results_df, pd.DataFrame({


In [21]:
best_results_df

,Model,Sampler,Sampling Strategy,F1 Score
0,AdaBoostClassifier,RandomUnderSampler,0.05,0.116667
1,DecisionTreeClassifier,RandomOverSampler,0.05,0.078947
2,GradientBoostingClassifier,RandomUnderSampler,0.10,0.124352
3,KNeighborsClassifier,SMOTE,0.05,0.036036
4,RandomForestClassifier,RandomUnderSampler,0.10,0.098361
5,SVC,RandomOverSampler,0.01,0.000000
6,XGBClassifier,RandomUnderSampler,0.05,0.112676


## Maintenant qu'on a une vue d'ensemble claire sur les méthodes de re-échantillonage qui marchent le mieux en fonction des modèles, on va reprendre ces combinaisons pour réaliser une optimisation cette fois-ci des hyper-paramètres des modèles à l'aide d'un greedsearch.

### Model: GradientBoostingClassifier - Sampler: RandomUnderSampler - Sampling Strategy: 0.1

In [35]:
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline

# Sous-échantillonnage
rus = RandomUnderSampler(sampling_strategy=0.1)
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

# Modèle de l'arbre de décision
gb_classifier = GradientBoostingClassifier()

# Pipeline
pipeline = Pipeline([
    ('classifier', gb_classifier)
])

# Créer un objet TimeSeriesSplit pour la cross-validation
tscv = TimeSeriesSplit(n_splits = 5)

# Paramètres pour la recherche sur la grille
param_grid = {
    'classifier__n_estimators': [50, 100],
    'classifier__max_depth': [5, 10, 15],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [2, 4]
}

# Recherche sur la grille
grid_search = GridSearchCV(pipeline, param_grid, scoring='f1', cv=tscv)
grid_search.fit(X_train, y_train)

 
# Afficher les meilleurs paramètres et le meilleur score F1
print("Meilleurs paramètres:", grid_search.best_params_)
print("Meilleur score F1:", grid_search.best_score_)

Meilleurs paramètres: {'classifier__max_depth': 15, 'classifier__min_samples_leaf': 2, 'classifier__min_samples_split': 5, 'classifier__n_estimators': 100}
Meilleur score F1: 0.10323029387708163


In [17]:
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline

# Sous-échantillonnage
ros = RandomOverSampler(sampling_strategy=0.1)
X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)

# Modèle de l'arbre de décision
gb_classifier = GradientBoostingClassifier()

# Pipeline
pipeline = Pipeline([
    ('classifier', gb_classifier)
])

# Créer un objet TimeSeriesSplit pour la cross-validation
tscv = TimeSeriesSplit(n_splits = 5)

# Paramètres pour la recherche sur la grille
param_grid = {
    'classifier__n_estimators': [50, 100],
    'classifier__max_depth': [5, 10, 15],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [2, 4]
}

# Recherche sur la grille
grid_search = GridSearchCV(pipeline, param_grid, scoring='f1', cv=tscv)
grid_search.fit(X_train_ros, y_train_ros)

 
# Afficher les meilleurs paramètres et le meilleur score F1
print("Meilleurs paramètres:", grid_search.best_params_)
print("Meilleur score F1:", grid_search.best_score_)

In [44]:
import numpy as np

# Compter le nombre d'éléments uniques dans y_train_rus
unique_classes, counts = np.unique(y_train_rus, return_counts=True)

# Afficher le nombre d'éléments uniques et leurs occurrences
for cls, count in zip(unique_classes, counts):
    print(f"Classe {cls}: {count} occurrences")

Classe 0: 2290 occurrences
Classe 1: 229 occurrences


In [33]:
len(y_train)

38827

In [34]:
len(y_train_rus)

2519

In [16]:
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline

# Sous-échantillonnage
rus = RandomUnderSampler(sampling_strategy=0.05)
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

# Modèle de l'AdaBoostClassifier
adab_classifier = AdaBoostClassifier(base_estimator=DecisionTreeClassifier())  # Utilise un arbre de décision faible par défaut

# Pipeline
pipeline = Pipeline([
    ('classifier', adab_classifier)
])

# Créer un objet TimeSeriesSplit pour la cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# Paramètres pour la recherche sur la grille
param_grid = {
    'classifier__n_estimators': [50, 100],
    'classifier__base_estimator__max_depth': [1, 2, 3],  # Ajustez ici pour le paramètre de l'arbre faible
    'classifier__learning_rate': [0.1, 0.5, 1.0],
}

# Recherche sur la grille
grid_search = GridSearchCV(pipeline, param_grid, scoring='f1', cv=tscv)
grid_search.fit(X_train, y_train)

# Afficher les meilleurs paramètres et le meilleur score F1
print("Meilleurs paramètres:", grid_search.best_params_)
print("Meilleur score F1:", grid_search.best_score_)


c:\Python311\Lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
c:\Python311\Lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
c:\Python311\Lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
c:\Python311\Lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
c:\Python311\Lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
c:\Python311\Lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` w

Meilleurs paramètres: {'classifier__base_estimator__max_depth': 1, 'classifier__learning_rate': 0.5, 'classifier__n_estimators': 100}
Meilleur score F1: 0.05411832448134206


In [30]:

# Utiliser le meilleur modèle trouvé par la recherche sur la grille
best_model = grid_search.best_estimator_

# Faire des prédictions sur l'ensemble de test
y_pred = best_model.predict(X_test)

# Calculer le F1-score sur l'ensemble de test
f1_test = f1_score(y_test, y_pred)

# Afficher le F1-score sur l'ensemble de test
print("F1 Score sur l'ensemble de test:", f1_test)


F1 Score sur l'ensemble de test: 0.054590570719602965
